In [2]:
import torch
import numpy as np
import os
# import torchaudio
# import librosa
# from concurrent.futures import ThreadPoolExecutor
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

print("imports successful!")

imports successful!


In [3]:
x_train = torch.load("/scratch/gd2574/AudioSet-classification/Data/train/train_rep.pt")
y_train = torch.load("/scratch/gd2574/AudioSet-classification/Data/train/labels.pt")
x_test = torch.load("/scratch/gd2574/AudioSet-classification/Data/test/test_rep.pt")
y_test = torch.load("/scratch/gd2574/AudioSet-classification/Data/test/labels.pt")

/state/partition1/job-54426358/ipykernel_3164381/3713021574.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_train = torch.load("/scratch/gd2574/AudioSet-classification

# Data loaders

In [3]:
def split_loader(full_dataset):
    total_size = len(full_dataset)
    train_size = (1 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size],generator=torch.Generator().manual_seed(42))
        
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

In [10]:
def create_stratified_split(x_data, y_data, random_state=42):
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    indices = np.arange(len(y_data))
    for train_idx, val_idx in mskf.split(indices, y_data):
        return train_idx, val_idx

# Helper functions

In [5]:
def calculate_map(y_true, y_pred):
    n_classes = y_true.shape[1]
    average_precisions = []
    for i in range(n_classes):
        y_true_class = y_true[:, i]
        y_pred_class = y_pred[:, i]
        ap = average_precision_score(y_true_class, y_pred_class)
        average_precisions.append(ap)
    average_precisions_sorted = sorted(average_precisions, reverse=True)
    print("\nTop 5 class-wise Average Precisions:")
    for i, ap in enumerate(average_precisions_sorted[:5]):
        print(f"Class {i+1}: {ap:.4f}")
    
    return np.mean(average_precisions)

# Train models

In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    print("Model being trained: ", model.__class__.__name__)
    model = model.to(device)
    best_map = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_predictions = []
        val_targets = []
        val_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                val_predictions.extend(outputs.cpu().numpy())
                val_targets.extend(batch_y.cpu().numpy())
        
        # lists -> numpy arrays for metric calculation
        val_predictions = np.array(val_predictions)
        val_targets = np.array(val_targets)
        val_map = calculate_map(val_targets, val_predictions)
        val_f1 = f1_score(val_targets, (val_predictions > 0.5).astype(float), average='micro')
        print(f'\nEpoch {epoch+1}/{num_epochs}:')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation MAP: {val_map:.4f}')
        print(f'Validation F1-Score: {val_f1:.4f}')
        
        # Save best model based on MAP
        if val_map > best_map:
            best_map = val_map
            model_name = model.__class__.__name__ 
            filename = f"best_{model_name}_map.pth"
            torch.save(model.state_dict(), filename)
            print(f"New best model saved as {filename}!")
            # torch.save(model.state_dict(), 'best_model_map.pth')
            # print("New best model saved!")
        
        print('-' * 50)
    return model, best_map  

# Test model

In [7]:
def test_model(model, device='cuda'):
    print("Model being tested: ", model.__class__.__name__)
    test_dataset = TensorDataset(x_test.float(), y_test.float())
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.load_state_dict(torch.load("best_model_map.pth"))
    model.eval()
    
    test_predictions = []
    test_targets = []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.float().to(device)
            outputs = model(batch_x)
            test_predictions.extend(outputs.cpu().numpy())
            test_targets.extend(batch_y.cpu().numpy())
    
    test_predictions = np.array(test_predictions)
    test_targets = np.array(test_targets)
    
    test_map = calculate_map(test_targets, test_predictions)
    test_f1 = f1_score(test_targets, (test_predictions > 0.5).astype(float), average="micro")
    
    print(f"Test MAP: {test_map:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    
    return test_map, test_f1

# CNN

In [8]:
class CNNClassifier(nn.Module):
    def __init__(self, input_channels=768, num_classes=527):
        super(CNNClassifier, self).__init__()
        
        # First Convolutional Block
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )   
        # Second Convolutional Block
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        # Third Convolutional Block
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.flatten_size = 64 * (496 // (2*2*2))
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x

# RNN

In [9]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size=768, hidden_size=256, num_layers=2, num_classes=527, dropout=0.5):
        super(RNNClassifier, self).__init__()
        
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, 1),
            nn.Tanh()
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )
        
    def attention_net(self, rnn_output):
        # rnn_output shape: (batch_size, seq_len, hidden_size * 2)
        attention_weights = self.attention(rnn_output)  # (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_weights, dim=1)
        context = torch.sum(attention_weights * rnn_output, dim=1)  # (batch_size, hidden_size * 2)
        return context
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        rnn_output, _ = self.rnn(x)  # (batch_size, seq_len, hidden_size * 2)
        # Apply attention
        context = self.attention_net(rnn_output)  # (batch_size, hidden_size * 2)
        # Final classification
        output = self.fc(context)  # (batch_size, num_classes)
        return output

In [15]:
# class RNNClassifier(nn.Module):
#     def __init__(self, input_size=768, hidden_size=512, num_layers=3, num_classes=527, dropout=0.3):
#         super(RNNClassifier, self).__init__()
        
#         # RNN layers
#         self.rnn = nn.RNN(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             dropout=dropout if num_layers > 1 else 0,
#             bidirectional=True
#         )
        
#         # Fully connected layers
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size * 2, 1024),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(512, num_classes)
#         )
        
#     def forward(self, x):
#         # print(x.shape)
#         rnn_output, _ = self.rnn(x)
#         # Average pooling over timesteps
#         context = torch.mean(rnn_output, dim=1)
#         # print(context.shape)
#         output = self.fc(context)
#         return output


In [16]:
batch_size = 32
learning_rate= 0.001
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train_float = x_train.float()
y_train_float = y_train.float()
full_dataset = TensorDataset(x_train_float, y_train_float)
total_size = len(full_dataset)

# USING STRATIFIED SPLITS
train_indices, val_indices = create_stratified_split(x_train_float, y_train_float.numpy())

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

criterion = nn.BCELoss()

In [17]:
model = RNNClassifier()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
model, best_val_map = train_model( model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Model being trained:  RNNClassifier
torch.Size([32, 496, 768])
torch.Size([32, 1024])


/opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [30,0,0], thread: [66,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [30,0,0], thread: [68,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [30,0,0], thread: [70,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [30,0,0], thread: [73,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [30,0,0], thread: [74,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1729647406761/work/at

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# GRU

In [10]:
class GRUClassifier(nn.Module):
    def __init__(self, input_size=768, hidden_size=256, num_layers=2, num_classes=527, dropout=0.5):
        super(GRUClassifier, self).__init__()
        
        # GRU layers
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, 1),
            nn.Tanh()
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )
        
    def attention_net(self, gru_output):
        # gru_output shape: (batch_size, seq_len, hidden_size * 2)
        attention_weights = self.attention(gru_output)  # (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_weights, dim=1)
        context = torch.sum(attention_weights * gru_output, dim=1)  # (batch_size, hidden_size * 2)
        return context
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        gru_output, _ = self.gru(x)  # (batch_size, seq_len, hidden_size * 2)
        # attention
        context = self.attention_net(gru_output)  # (batch_size, hidden_size * 2)
        # final classification
        output = self.fc(context)  # (batch_size, num_classes)
        return output

# LSTM

In [11]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=768, hidden_size=256, num_layers=2, num_classes=527, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention mechanism (reusing the same structure)
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, 1),
            nn.Tanh()
        )
        
        # Layer Normalization
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        
        # Fully connected layers with residual connections
        self.fc1 = nn.Linear(hidden_size * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        
        # Additional components
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def attention_net(self, lstm_output):
        # lstm_output shape: (batch_size, seq_len, hidden_size * 2)
        attention_weights = self.attention(lstm_output)
        attention_weights = torch.softmax(attention_weights, dim=1)
        context = torch.sum(attention_weights * lstm_output, dim=1)
        return context, attention_weights
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        
        # LSTM forward pass
        lstm_output, (hidden, cell) = self.lstm(x)
        
        # Apply layer normalization
        lstm_output = self.layer_norm(lstm_output)
        
        # Apply attention mechanism
        context, attention_weights = self.attention_net(lstm_output)
        
        # Fully connected layers with residual connections
        out = self.fc1(context)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        # Final classification layer
        out = self.fc3(out)
        out = self.sigmoid(out)
        
        return out

# Training setup and hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_lstm_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)
    best_map = 0
    best_model_path = 'best_lstm_model.pth'
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_predictions = []
        val_targets = []
        val_loss = 0
        
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                
                val_predictions.extend(outputs.cpu().numpy())
                val_targets.extend(batch_y.cpu().numpy())
        
        # Calculate metrics
        val_predictions = np.array(val_predictions)
        val_targets = np.array(val_targets)
        val_map = calculate_map(val_targets, val_predictions)
        val_f1 = f1_score(val_targets, (val_predictions > 0.5).astype(float), average='micro')
        
        # Print epoch results
        print(f'\nEpoch {epoch+1}/{num_epochs}:')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation MAP: {val_map:.4f}')
        print(f'Validation F1-Score: {val_f1:.4f}')
        
        # Save best model
        if val_map > best_map:
            best_map = val_map
            torch.save(model.state_dict(), best_model_path)
            print("New best model saved!")
        
        print('-' * 50)
    
    return model

# lessgooo

In [12]:
batch_size = 32
learning_rate= 0.001
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train_float = x_train.float()
y_train_float = y_train.float()
full_dataset = TensorDataset(x_train_float, y_train_float)
total_size = len(full_dataset)

# USING STRATIFIED SPLITS
train_indices, val_indices = create_stratified_split(x_train_float, y_train_float.numpy())

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

criterion = nn.BCELoss()

In [17]:
# CNN

In [13]:
model = CNNClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model, best_val_map = train_model( model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Model being trained:  CNNClassifier

Top 5 class-wise Average Precisions:
Class 1: 0.7869
Class 2: 0.7792
Class 3: 0.7717
Class 4: 0.7549
Class 5: 0.7542

Epoch 1/5:
Training Loss: 0.0282
Validation Loss: 0.0187
Validation MAP: 0.1309
Validation F1-Score: 0.2431
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.9102
Class 2: 0.8789
Class 3: 0.8782
Class 4: 0.8570
Class 5: 0.8563

Epoch 2/5:
Training Loss: 0.0185
Validation Loss: 0.0156
Validation MAP: 0.2461
Validation F1-Score: 0.3313
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.9760
Class 2: 0.9393
Class 3: 0.9175
Class 4: 0.9159
Class 5: 0.9085

Epoch 3/5:
Training Loss: 0.0161
Validation Loss: 0.0143
Validation MAP: 0.3072
Validation F1-Score: 0.3626
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 1.0000
Class 2: 0

In [14]:
print("Model being tested: ", model.__class__.__name__)
test_map, test_f1 = test_model(model, device)

Model being tested:  CNNClassifier
Model being tested:  CNNClassifier


/state/partition1/job-54426358/ipykernel_3159100/3232669007.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_map.pth"))



Top 5 class-wise Average Precisions:
Class 1: 0.8946
Class 2: 0.8945
Class 3: 0.8862
Class 4: 0.8677
Class 5: 0.8260
Test MAP: 0.3027
Test F1-Score: 0.3985


In [ ]:
# RNN

In [15]:
model = RNNClassifier()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
model, best_val_map = train_model( model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Model being trained:  RNNClassifier

Top 5 class-wise Average Precisions:
Class 1: 0.2838
Class 2: 0.2168
Class 3: 0.1102
Class 4: 0.0492
Class 5: 0.0481

Epoch 1/5:
Training Loss: 0.0340
Validation Loss: 0.0254
Validation MAP: 0.0068
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.2775
Class 2: 0.2099
Class 3: 0.1456
Class 4: 0.1333
Class 5: 0.1147

Epoch 2/5:
Training Loss: 0.0270
Validation Loss: 0.0253
Validation MAP: 0.0083
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.2666
Class 2: 0.2277
Class 3: 0.1333
Class 4: 0.1213
Class 5: 0.1068

Epoch 3/5:
Training Loss: 0.0264
Validation Loss: 0.0302
Validation MAP: 0.0089
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.2888
Class 2: 0

In [16]:
print("Model being tested: ", model.__class__.__name__)
test_map, test_f1 = test_model(model, device)

Model being tested:  RNNClassifier
Model being tested:  RNNClassifier


/state/partition1/job-54426358/ipykernel_3159100/3232669007.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_map.pth"))



Top 5 class-wise Average Precisions:
Class 1: 0.3004
Class 2: 0.2662
Class 3: 0.1148
Class 4: 0.0570
Class 5: 0.0567
Test MAP: 0.0080
Test F1-Score: 0.0000


In [ ]:
# GRU

In [17]:
model = GRUClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model, best_val_map = train_model( model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Model being trained:  GRUClassifier

Top 5 class-wise Average Precisions:
Class 1: 0.2328
Class 2: 0.2225
Class 3: 0.1408
Class 4: 0.0860
Class 5: 0.0832

Epoch 1/5:
Training Loss: 0.0351
Validation Loss: 0.0253
Validation MAP: 0.0076
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.4186
Class 2: 0.4179
Class 3: 0.4062
Class 4: 0.3689
Class 5: 0.2572

Epoch 2/5:
Training Loss: 0.0267
Validation Loss: 0.0251
Validation MAP: 0.0126
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.4824
Class 2: 0.3382
Class 3: 0.2522
Class 4: 0.2498
Class 5: 0.2354

Epoch 3/5:
Training Loss: 0.0260
Validation Loss: 0.0243
Validation MAP: 0.0178
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.7218
Class 2: 0

In [18]:
print("Model being tested: ", model.__class__.__name__)
test_map, test_f1 = test_model(model, device)

Model being tested:  GRUClassifier
Model being tested:  GRUClassifier


/state/partition1/job-54426358/ipykernel_3159100/3232669007.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_map.pth"))



Top 5 class-wise Average Precisions:
Class 1: 0.7507
Class 2: 0.6778
Class 3: 0.6686
Class 4: 0.6627
Class 5: 0.5654
Test MAP: 0.0582
Test F1-Score: 0.1950


In [ ]:
# LSTM

In [19]:
model = LSTMClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model, best_val_map = train_model( model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Model being trained:  LSTMClassifier

Top 5 class-wise Average Precisions:
Class 1: 0.3542
Class 2: 0.3502
Class 3: 0.3441
Class 4: 0.3351
Class 5: 0.2450

Epoch 1/5:
Training Loss: 0.0321
Validation Loss: 0.0258
Validation MAP: 0.0121
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.2485
Class 2: 0.2201
Class 3: 0.2112
Class 4: 0.2101
Class 5: 0.2059

Epoch 2/5:
Training Loss: 0.0265
Validation Loss: 0.0252
Validation MAP: 0.0123
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.3881
Class 2: 0.3529
Class 3: 0.3240
Class 4: 0.3033
Class 5: 0.2542

Epoch 3/5:
Training Loss: 0.0259
Validation Loss: 0.0239
Validation MAP: 0.0185
Validation F1-Score: 0.0000
New best model saved!
--------------------------------------------------

Top 5 class-wise Average Precisions:
Class 1: 0.6917
Class 2: 

In [20]:
print("Model being tested: ", model.__class__.__name__)
test_map, test_f1 = test_model(model, device)

Model being tested:  LSTMClassifier
Model being tested:  LSTMClassifier


/state/partition1/job-54426358/ipykernel_3159100/3232669007.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_map.pth"))



Top 5 class-wise Average Precisions:
Class 1: 0.7796
Class 2: 0.7755
Class 3: 0.7537
Class 4: 0.7502
Class 5: 0.7455
Test MAP: 0.0659
Test F1-Score: 0.1913
